In [10]:
import pandas as pd
import gensim
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [29]:
%store -r training_padded_w2v
%store -r testing_padded_w2v

%store -r glove_training_padded
%store -r glove_testing_padded

%store -r training_padded_bert
%store -r testing_padded_bert

%store -r training_padded_elmo
%store -r testing_padded_elmo


In [30]:
%store -r y_train
%store -r y_test

In [44]:
concat_ensemble_train = [training_padded_w2v, training_padded_bert,  glove_training_padded]
concat_ensemble_test = [testing_padded_w2v, testing_padded_bert,  glove_testing_padded]

In [43]:
len(concat_ensemble_train[0])

400

In [39]:
concat_ensemble_train = (concat_ensemble_train[0]  + concat_ensemble_train[1] + concat_ensemble_train[2]) / 3

In [40]:
concat_ensemble_test = (concat_ensemble_test[0] + concat_ensemble_test[1] + concat_ensemble_test[2]) / 3

In [32]:
training_padded = pad_sequences(concat_ensemble_train, maxlen=120, truncating='post')
testing_padded = pad_sequences(concat_ensemble_test, maxlen=120, truncating='post')

In [35]:
training_padded.shape

(3, 120, 120, 300)

In [41]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(concat_ensemble_train, y_train, epochs=num_epochs,batch_size=batch, validation_data=(concat_ensemble_test, y_test))
model.summary()

Epoch 1/10
13/13 [==============================] - 1s 17ms/step - loss: 0.6932 - accuracy: 0.5350 - recall: 0.0604 - auc: 0.4812 - precision: 0.4231 - false_negatives: 171.0000 - false_positives: 15.0000 - val_loss: 0.6904 - val_accuracy: 0.5200 - val_recall: 0.0625 - val_auc: 0.5711 - val_precision: 0.5000 - val_false_negatives: 45.0000 - val_false_positives: 3.0000
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 0.6281 - accuracy: 0.7625 - recall: 0.4945 - auc: 0.9385 - precision: 0.9677 - false_negatives: 92.0000 - false_positives: 3.0000 - val_loss: 0.6921 - val_accuracy: 0.5000 - val_recall: 0.0208 - val_auc: 0.5647 - val_precision: 0.2500 - val_false_negatives: 47.0000 - val_false_positives: 3.0000
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 0.5625 - accuracy: 0.8700 - recall: 0.7143 - auc: 0.9947 - precision: 1.0000 - false_negatives: 52.0000 - false_positives: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.5000 - val_recall:

In [51]:
meta_array_train = []
meta_array_test = []

for val in range(400):
    if val < 150:
        meta_array_train.append(concat_ensemble_train[0][val])
    elif val < 300:
        meta_array_train.append(concat_ensemble_train[1][val])
    else:
        meta_array_train.append(concat_ensemble_train[2][val])

for val in range(100):
    if val < 38:
        meta_array_test.append(concat_ensemble_test[0][val])
    elif val < 76:
        meta_array_test.append(concat_ensemble_test[1][val])
    else:
        meta_array_test.append(concat_ensemble_test[2][val])

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0

In [54]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(meta_array_train, y_train, epochs=num_epochs,batch_size=batch, validation_data=(meta_array_test, y_test))
model.summary()

ValueError: Data cardinality is ambiguous:
  x sizes: 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 1